In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './credentials/google_application_credentials.json'

In [2]:
from credentials import api_keys

In [3]:
import io
import time
import requests
import numpy as np
import pandas as pd
from sklearn import metrics
from google.cloud import vision
from tqdm.notebook import tqdm
from tqdm.notebook import trange, tqdm

## Data Preprocessing

In [4]:
df1 = pd.read_csv('csv_3000_hate/sample_3000_labeled.csv')
df2 = pd.read_csv('csv_2000_no_politics/sample_2000_labeled.csv')

In [5]:
df1 = df1[['id','sample_path','label']]
df2 = df2[['id','sample_path','label']]


In [6]:
# combine 50,000 images
df = df1.append(df2)
# shuffle the images
df = df.sample(frac = 1).reset_index(drop=True)

In [7]:
df['label'] = df['label'].apply(int)

In [8]:
# change column path to 'img'
df = df.rename(columns={'sample_path': 'img'})

In [9]:
print(df.shape)

(5000, 3)


## Google Cloud Vision API

In [10]:
def google_vision_api_without_spoof(path):
    """Google Cloud Vision API"""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.safe_search_detection(image=image)
    # print(response)
    safe = response.safe_search_annotation

    if response.error.message:
        raise Exception(response.error.message)

    # if safe.adult > 3 or safe.medical > 3 or safe.violence > 3 or safe.racy > 3:
    # #     print('Image: {}, UNSAFE'.format(path))
    #     return 1
    # else:
    # #     print('Image: {}, SAFE'.format(path))
    #     return 0

    #list all safe search labels:
  
    # print(safe.adult)
    # print(safe.medical)
    # print(safe.violence)
    # print(safe.racy)
    return(int(safe.adult),int(safe.medical),int(safe.violence),int(safe.racy))

In [11]:
df_google = df.copy()

tqdm.pandas()
df_google['label_pred'] = df_google['img'].progress_apply(google_vision_api_without_spoof)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [75]:
df_google['label_pred'] = df_google['label_pred'].astype('str')
df_google[['pred_adult','pred_medical','pred_viol','pred_racy']] = df_google['label_pred'].str.split(',',expand=True)

KeyError: 'label_pred'

In [15]:
df_google['pred_adult'] = df_google['pred_adult'].replace(to_replace='\(', value="", regex=True)
df_google['pred_racy'] = df_google['pred_racy'].replace(to_replace='\)', value="", regex=True)
df_google['pred_adult'] = df_google['pred_adult'].apply(int)
df_google['pred_medical'] = df_google['pred_medical'].apply(int)
df_google['pred_viol'] = df_google['pred_viol'].apply(int)
df_google['pred_racy'] = df_google['pred_racy'].apply(int)

In [19]:
df_google = df_google.drop(columns=['label_pred'])
# print(df_google)

        id                                           img  label  pred_adult  \
0     1438  ./annot_2000_no_politics/EQw2J_mX0AAxvoq.jpg      1           1   
1      172  ./annot_2000_no_politics/EWAuxVaWkAIv7Nv.jpg      0           1   
2     2156         ./annot_3000_hate/EeoBkhCXoAAc5IQ.jpg      0           2   
3     2749         ./annot_3000_hate/EgbTs6AWsAAHAj9.jpg      1           1   
4      738  ./annot_2000_no_politics/E0KvNiSVIAQ4vZR.jpg      1           1   
...    ...                                           ...    ...         ...   
4995   853         ./annot_3000_hate/EfQTejhX0AIt7Hp.jpg      1           1   
4996  1291  ./annot_2000_no_politics/EoOmgneWEAAkSy3.jpg      0           1   
4997  2136         ./annot_3000_hate/Et4HU8vUcAEY7y5.jpg      0           1   
4998   516         ./annot_3000_hate/EUt652pUYAEkpdI.jpg      0           1   
4999  1540  ./annot_2000_no_politics/EeqFVFTXYAA3Bs1.jpg      0           1   

      pred_medical  pred_viol  pred_racy  
0       

In [20]:
# save the result to './results/df_google.csv'
df_google.to_csv('./results/df_google.csv')
print(df_google)

        id                                           img  label  pred_adult  \
0     1438  ./annot_2000_no_politics/EQw2J_mX0AAxvoq.jpg      1           1   
1      172  ./annot_2000_no_politics/EWAuxVaWkAIv7Nv.jpg      0           1   
2     2156         ./annot_3000_hate/EeoBkhCXoAAc5IQ.jpg      0           2   
3     2749         ./annot_3000_hate/EgbTs6AWsAAHAj9.jpg      1           1   
4      738  ./annot_2000_no_politics/E0KvNiSVIAQ4vZR.jpg      1           1   
...    ...                                           ...    ...         ...   
4995   853         ./annot_3000_hate/EfQTejhX0AIt7Hp.jpg      1           1   
4996  1291  ./annot_2000_no_politics/EoOmgneWEAAkSy3.jpg      0           1   
4997  2136         ./annot_3000_hate/Et4HU8vUcAEY7y5.jpg      0           1   
4998   516         ./annot_3000_hate/EUt652pUYAEkpdI.jpg      0           1   
4999  1540  ./annot_2000_no_politics/EeqFVFTXYAA3Bs1.jpg      0           1   

      pred_medical  pred_viol  pred_racy  
0       

In [11]:
df_test = pd.read_csv('./results/df_google.csv')
print(df_test)

      Unnamed: 0    id                                           img  label  \
0              0  1438  ./annot_2000_no_politics/EQw2J_mX0AAxvoq.jpg      1   
1              1   172  ./annot_2000_no_politics/EWAuxVaWkAIv7Nv.jpg      0   
2              2  2156         ./annot_3000_hate/EeoBkhCXoAAc5IQ.jpg      0   
3              3  2749         ./annot_3000_hate/EgbTs6AWsAAHAj9.jpg      1   
4              4   738  ./annot_2000_no_politics/E0KvNiSVIAQ4vZR.jpg      1   
...          ...   ...                                           ...    ...   
4995        4995   853         ./annot_3000_hate/EfQTejhX0AIt7Hp.jpg      1   
4996        4996  1291  ./annot_2000_no_politics/EoOmgneWEAAkSy3.jpg      0   
4997        4997  2136         ./annot_3000_hate/Et4HU8vUcAEY7y5.jpg      0   
4998        4998   516         ./annot_3000_hate/EUt652pUYAEkpdI.jpg      0   
4999        4999  1540  ./annot_2000_no_politics/EeqFVFTXYAA3Bs1.jpg      0   

      pred_adult  pred_medical  pred_viol  pred_rac

### Define the safety criteria

In [14]:
# if safe.adult > 3 or safe.medical > 3 or safe.violence > 3 or safe.racy > 3, we record it as 1 which means unsafe
df_test['pre_label'] = df_test[['pred_adult','pred_medical','pred_viol','pred_racy']].max(axis=1)
df_test['pre_label'] = df_test['pre_label'].apply(lambda x: 1 if x>3 else 0)
# check the number of unsafe labels from google
df_test['pre_label'].value_counts()

0    4850
1     150
Name: pre_label, dtype: int64

### Get the report

In [13]:
print('Classification Report for Google Cloud Vision API:')
target_names = ['safe', 'not-safe']
print(metrics.classification_report(df_test['label'], df_test['pre_label'], target_names=target_names, digits=4))

Classification Report for Google Cloud Vision API:
              precision    recall  f1-score   support

        safe     0.7332    0.9719    0.8358      3659
    not-safe     0.3133    0.0350    0.0630      1341

    accuracy                         0.7206      5000
   macro avg     0.5233    0.5034    0.4494      5000
weighted avg     0.6206    0.7206    0.6286      5000



## Yahoo Open NSFW

In [5]:
print('Classification Report for Yahoo Open NSFW:')
df_yahoo = pd.read_csv('./results/df_yahoo.csv')
df_yahoo['pred_yh'] = df_yahoo['pred_yh_value'].apply(lambda x: 1 if x >= 0.5 else 0 )
target_names = ['safe', 'not-safe']
print(metrics.classification_report(df_yahoo['label'], df_yahoo['pred_yh'], target_names=target_names, digits=4))

Classification Report for Yahoo Open NSFW:
              precision    recall  f1-score   support

        safe     0.5051    0.9802    0.6667       253
    not-safe     0.4444    0.0162    0.0312       247

    accuracy                         0.5040       500
   macro avg     0.4748    0.4982    0.3490       500
weighted avg     0.4751    0.5040    0.3528       500



## Clarifai NSFW API

In [11]:
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_pb2, status_code_pb2

channel = ClarifaiChannel.get_json_channel()
stub = service_pb2_grpc.V2Stub(channel)

metadata = (('authorization', api_keys.CLARIFAI_API_KEY),)


In [5]:
def clarifai_api(path):
    with open(path, "rb") as f:
        file_bytes = f.read()

    request = service_pb2.PostModelOutputsRequest(
        # This is the model ID of a publicly available General model. 
        # You may use any other public or custom model ID.
        model_id='e9576d86d2004ed1a38ba0cf39ecb4b1',
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        base64=file_bytes
                    )
                )
            )
        ])
    response = stub.PostModelOutputs(request, metadata=metadata)

    if response.status.code != status_code_pb2.SUCCESS:
        raise Exception("Post model outputs failed, status: " + response.status.description)

    # for concept in response.outputs[0].data.concepts:
    #     print('%12s: %.2f' % (concept.name, concept.value))

    # if response.outputs[0].data.concepts[1].value >= 0.4:
    #     print('Image: {}, Offensive: {:.2f}'.format(path, response.outputs[0].data.concepts[1].value))
    # else:
    #     print('Image: {}, not-Offensive: {:.2f}'.format(path, response.outputs[0].data.concepts[1].value))

    return response.outputs[0].data.concepts[1].name, response.outputs[0].data.concepts[1].value

In [6]:
# df_clarifai = df.copy()
# header = ["id", "img", "label"]
# df_clarifai.to_csv('./df_clarifai.csv', columns = header)
# print(df_clarifai)

In [6]:
df_c = pd.read_csv('./df_clarifai.csv')
df_c = df_c.loc[:, ~df_c.columns.str.contains('^Unnamed')]
#print(df_c)

In [7]:
df_c1 = df_c[:1000]
df_c2 = df_c[1000:2000]
df_c3 = df_c[2000:3000]
df_c4 = df_c[3000:4000]
df_c5 = df_c[4000:]
#df_c5

In [12]:
tqdm.pandas()
df_c1['pred_cf'] = df_c1['img'].progress_apply(clarifai_api)
# df_c1.to_csv('./results/df_clarifai.csv', mode='a', header=False)

  0%|          | 0/1000 [00:00<?, ?it/s]

UsageError: Please provide metadata with the format of (('authorization', 'Key YOUR_CLARIFAI_API_KEY'),)

In [8]:
print('Classification Report for Clarifai NSFW API:')
df_clarifai = pd.read_csv('./results/df_clarifai.csv')
df_clarifai['pred_cf'] = df_clarifai['pred_cf_value'].apply(lambda x: 1 if x >= 0.5 else 0 )
target_names = ['safe', 'not-safe']
print(metrics.classification_report(df_clarifai['label'], df_clarifai['pred_cf'], target_names=target_names, digits=4))

Classification Report for Clarifai NSFW API:


FileNotFoundError: [Errno 2] No such file or directory: './results/df_clarifai.csv'

## DeepAI API

In [10]:
def deep_ai_api(path):
    deep_ai_request = requests.post(
        'https://api.deepai.org/api/content-moderation',
        files={
            'image': open(path, 'rb'),
        },
        headers={'api-key': api_keys.DEEPAI_API_KEY}
    )
    # print(deep_ai_request.json())
    nsfw_score = deep_ai_request.json()['output']['nsfw_score']
#    print('Image: {}, nsfw_score: {:.2f}'.format(path, nsfw_score))
    return nsfw_score

In [11]:
df_deepai = df.copy()

tqdm.pandas()
df_deepai['pred_da_value'] = df_deepai['img'].progress_apply(deep_ai_api)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [12]:
df_deepai.to_csv('./results/df_deepai.csv')
print(df_deepai)

        id                                           img  label  pred_da_value
0     1031  ./annot_2000_no_politics/EzVXJPlVgAQOich.jpg      0       0.015985
1      661         ./annot_3000_hate/EtO03ozUUAA8OPx.jpg      0       0.024578
2      886  ./annot_2000_no_politics/EVXX4kGX0AAeG-u.jpg      0       0.000871
3      378         ./annot_3000_hate/EUkr_PrUwAAgFTy.jpg      0       0.001544
4     1288         ./annot_3000_hate/EecEF9eX0AAZ1Un.jpg      0       0.697362
...    ...                                           ...    ...            ...
4995  2223         ./annot_3000_hate/Ee4mZF_VAAAUGRq.jpg      1       0.063505
4996  2375         ./annot_3000_hate/Eyx-2k-VoAMDtTU.jpg      1       0.014567
4997   901  ./annot_2000_no_politics/EemqXurWsAECWL4.jpg      1       0.006366
4998  1126  ./annot_2000_no_politics/EV7jt3KXYAUCEJ7.jpg      0       0.001977
4999   350  ./annot_2000_no_politics/EgH1jIwU8AAwlcl.jpg      0       0.001412

[5000 rows x 4 columns]


In [13]:
print('Classification Report for DeepAI API:')
df_deepai = pd.read_csv('./results/df_deepai.csv')
df_deepai['pred_da'] = df_deepai['pred_da_value'].apply(lambda x: 1 if x >= 0.5 else 0 )
target_names = ['safe', 'not-safe']
print(metrics.classification_report(df_deepai['label'], df_deepai['pred_da'], target_names=target_names, digits=4))

Classification Report for DeepAI API:
              precision    recall  f1-score   support

        safe     0.7319    0.9932    0.8428      3659
    not-safe     0.2857    0.0075    0.0145      1341

    accuracy                         0.7288      5000
   macro avg     0.5088    0.5003    0.4286      5000
weighted avg     0.6123    0.7288    0.6206      5000



In [38]:
def deep_ai_api_object(path):
    deep_ai_request = requests.post(
        'https://api.deepai.org/api/content-moderation',
        files={
            'image': open(path, 'rb'),
        },
        headers={'api-key': api_keys.DEEPAI_API_KEY}
    )
    # print(deep_ai_request.json())
    # print(deep_ai_request.json()['output']['detections'])
    # print(len(deep_ai_request.json()['output']['detections']))
    objects = len(deep_ai_request.json()['output']['detections'])

    print('Image: {}, not-safe objects: {:>3}'.format(path, objects))
    return objects

In [39]:
df_deepai_objects = df.copy()
df_deepai_objects['pred_da_value'] = df_deepai_objects['img'].apply(deep_ai_api_object)
df_deepai_objects.to_csv('./results/df_deepai_objects.csv')
print(df_deepai_objects)

/img/92738.png, not-safe objects:   0
Image: ./data/img/53027.png, not-safe objects:   0
Image: ./data/img/27195.png, not-safe objects:   1
Image: ./data/img/43910.png, not-safe objects:   0
Image: ./data/img/05213.png, not-safe objects:   0
Image: ./data/img/73945.png, not-safe objects:   0
Image: ./data/img/56207.png, not-safe objects:   0
Image: ./data/img/68459.png, not-safe objects:   0
Image: ./data/img/63175.png, not-safe objects:   0
Image: ./data/img/82945.png, not-safe objects:   1
Image: ./data/img/34975.png, not-safe objects:   0
Image: ./data/img/61973.png, not-safe objects:   0
Image: ./data/img/16420.png, not-safe objects:   0
Image: ./data/img/73069.png, not-safe objects:   0
Image: ./data/img/47056.png, not-safe objects:   1
Image: ./data/img/56428.png, not-safe objects:   0
Image: ./data/img/54206.png, not-safe objects:   0
Image: ./data/img/93172.png, not-safe objects:   0
Image: ./data/img/56980.png, not-safe objects:   0
Image: ./data/img/79615.png, not-safe object

In [8]:
print('Classification Report for DeepAI API on Unsafe Content:')
df_deepai_objects = pd.read_csv('./results/df_deepai_objects.csv')
df_deepai_objects['pred_da'] = df_deepai_objects['pred_da_value'].apply(lambda x: 1 if x > 0 else 0 )
target_names = ['safe', 'not-safe']
print(metrics.classification_report(df_deepai_objects['label'], df_deepai_objects['pred_da'], target_names=target_names, digits=4))

Classification Report for DeepAI API on Unsafe Content:
              precision    recall  f1-score   support

        safe     0.5075    0.9368    0.6583       253
    not-safe     0.5152    0.0688    0.1214       247

    accuracy                         0.5080       500
   macro avg     0.5113    0.5028    0.3899       500
weighted avg     0.5113    0.5080    0.3931       500



## Amazon Rekognition

In [9]:
import boto3


In [10]:
def amz_reko_api_content(path):
    with open(path, "rb") as f:
        file_bytes = f.read()

    client=boto3.client('rekognition')

    response = client.detect_moderation_labels(Image={'Bytes': file_bytes})

    if len(response['ModerationLabels']) == 0:
        #print('Image: {}, SAFE'.format(path))
        return 0

    for item in response['ModerationLabels']:
        if item['Name'] == 'Hate Symbols':
            #print('Image: {}, Hateful'.format(path))
            return 1

    return 0

In [11]:
df_amz_hateful = df.copy()

tqdm.pandas()
df_amz_hateful['pred_amz_value'] = df_amz_hateful['img'].progress_apply(amz_reko_api_content)
df_amz_hateful.to_csv('./results/df_amz_hateful.csv')
print(df_amz_hateful)

  0%|          | 0/5000 [00:00<?, ?it/s]

        id                                           img  label  \
0     1770  ./annot_2000_no_politics/EeVg4ulU4AA8-W-.jpg      0   
1     2629         ./annot_3000_hate/EWfB6o6VcAErZAx.jpg      1   
2      719  ./annot_2000_no_politics/EVXYkh-WoAEkrf_.jpg      0   
3     1503         ./annot_3000_hate/EoRblq-UUAEAz-y.jpg      0   
4     1001         ./annot_3000_hate/EU2p99sU4AA58Xa.jpg      0   
...    ...                                           ...    ...   
4995   412         ./annot_3000_hate/EV6TFCTWAAAaNjD.jpg      0   
4996  2507         ./annot_3000_hate/EU0sh5SU8AAOsbn.jpg      0   
4997  1087  ./annot_2000_no_politics/EqOqshhXMAMdIup.jpg      0   
4998  1653  ./annot_2000_no_politics/EfESdZeUcAAH1AK.jpg      0   
4999  1451         ./annot_3000_hate/EgJk2qqXkAAg8YP.jpg      0   

      pred_amz_value  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0  
...              ...  
4995               0  
4996       

In [12]:
print('Classification Report for Amazon Rekognition on Hate Content:')
df_amz_hateful = pd.read_csv('./results/df_amz_hateful.csv')
df_amz_hateful['pred_amz'] = df_amz_hateful['pred_amz_value'].apply(lambda x: 1 if x >= 0.5 else 0 )
target_names = ['safe', 'hateful']
print(metrics.classification_report(df_amz_hateful['label'], df_amz_hateful['pred_amz'], target_names=target_names, digits=4))

Classification Report for Amazon Rekognition on Hate Content:
              precision    recall  f1-score   support

        safe     0.7331    0.9932    0.8435      3659
     hateful     0.4186    0.0134    0.0260      1341

    accuracy                         0.7304      5000
   macro avg     0.5759    0.5033    0.4348      5000
weighted avg     0.6488    0.7304    0.6243      5000



## Data Combination

In [10]:
df['pred_gv'] = df_google_2['pred_gv']
df['pred_yh'] = df_yahoo['pred_yh']
df['pred_cf'] = df_clarifai['pred_cf']
df['pred_da'] = df_deepai['pred_da']
df['pred_amz'] = df_amz['pred_amz']

In [11]:
df['pred_total'] = df['pred_gv'] + df['pred_yh'] + df_clarifai['pred_cf'] + df_deepai['pred_da'] + df_amz['pred_amz']
df

,id,img,label,pred_gv,pred_yh,pred_cf,pred_da,pred_amz,pred_total
0,8291,./data/img/08291.png,1,0,0,0,0,0,0
1,46971,./data/img/46971.png,1,0,0,0,0,0,0
2,3745,./data/img/03745.png,1,0,0,0,0,0,0
3,83745,./data/img/83745.png,1,1,1,0,1,0,3
4,80243,./data/img/80243.png,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
495,83675,./data/img/83675.png,0,0,0,0,0,0,0
496,37198,./data/img/37198.png,0,0,0,0,0,0,0
497,48670,./data/img/48670.png,0,0,0,0,0,0,0
498,9863,./data/img/09863.png,0,0,0,0,0,0,0


In [12]:
df_pos = df[df.label.eq(1)]
df_pos_rows = df_pos.shape[0]
print(df_pos_rows)
df_false_neg = df_pos[df_pos.pred_total.eq(0)]
df_false_neg_rows = df_false_neg.shape[0]
print(df_false_neg_rows)
pass_rate = df_false_neg_rows / df_pos_rows
print(pass_rate)

247
207
0.8380566801619433


In [14]:
import os
from shutil import copyfile
dst_path = './data/image_hateful_bypass/'
for src_path in df_false_neg['img']:
    filename = os.path.basename(src_path)
    dst = os.path.join(dst_path, filename)
    copyfile(src_path, dst)

In [15]:
df_neg = df[df.label.eq(0)]
df_neg_rows = df_neg.shape[0]
print(df_neg_rows)
df_false_pos = df_neg[df_neg.pred_total.gt(0)]
df_false_pos_rows = df_false_pos.shape[0]
print(df_false_pos_rows)
false_rate = df_false_pos_rows / df_neg_rows
print(false_rate)

253
37
0.14624505928853754


In [16]:
import os
from shutil import copyfile
dst_path = './data/image_nonhateful_mislabeled/'
for src_path in df_false_pos['img']:
    filename = os.path.basename(src_path)
    dst = os.path.join(dst_path, filename)
    copyfile(src_path, dst)